In [1]:
%load_ext autoreload
%autoreload 2

from pygeodyn.PYGEODYN import Pygeodyn
from gc import collect as gc_collect

# ## Initialization timeperiod
### Satellites:
Sat_List = ['spire083',
            #,'spire084',
            #'spire085',
           ]
import datetime


scaling_cadence = 3
scale_cadence = scaling_cadence
index_buffarc_start = 0
run_list = [
#                 'msis2',
#                 'jb2008',
                'dtm2020_o',
           ]

month_list = ['nov']
# month_list = ['oct', 'nov', 'dec', 'jan', 'feb', 'mar', 'apr']




dir_modeldat='/data/SatDragModelValidation/data/inputs/atmos_models'
run_dict={}
for i in run_list:
    for imonth,month in enumerate(month_list):
        if i =='msis2':
            run_dict[month+i]={}
            run_dict[month+i]['num'] = 5
            run_dict[month+i]['model_path'] = None
        if i =='dtm2020_o':
            run_dict[month+i]={}
            run_dict[month+i]['num'] = 3
            run_dict[month+i]['model_path'] = None
        if i =='jb2008':
            run_dict[month+i]={}
            run_dict[month+i]['num'] = 1
            run_dict[month+i]['model_path'] = None

print(run_dict)

{'novdtm2020_o': {'num': 3, 'model_path': None}}


In [2]:
import pandas as pd
from  datetime import datetime, timedelta
import pandas as pd
import numpy  as np
import linecache

def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

# for i,val in enumerate( timeavg_gfo):
#     date_near = nearest(timeavg_ice, val)
#     indx = np.where(pd.to_datetime(timeavg_ice) == pd.to_datetime(date_near))[0][0]
    
#     if np.abs((val - timeavg_ice[indx]).total_seconds()) > 5700:  # only compute % change if values within 1 orbit (95mins)
#         pass



def get_dates(test_timeperiod, file_raw_ICs):
    
    xyzline = pd.read_csv(
                    file_raw_ICs,
                    skiprows=23,
                #     nrows=1,
                    sep='\s+',
                    dtype=str,
                    names=[
                        'DateYMD',
                        'DateHMS',
                        'X',
                        'Y',
                        'Z',
                        'X_dot',
                        'Y_dot',
                        'Z_dot',
                    ],
                )
    date = pd.to_datetime(\
                        + xyzline['DateYMD']  \
                        + xyzline['DateHMS'], \
                                format='%Y-%m-%d%H:%M:%S')
    xyzline.insert(0, 'Date', date)
    del xyzline['DateYMD'], xyzline['DateHMS'], date
    del xyzline['X'], xyzline['Y'], xyzline['Z']
    del xyzline['X_dot'], xyzline['Y_dot'], xyzline['Z_dot']
    
#     list_arcs = []
#     start_0ut = []
#     end___0ut = []
#     start_update = []
    
    list_arc_startDT   = []
    list_epoch_startDT = []
    list_start_0ut     = []
    list_end___0ut     = []
    list_epoch_endDT   = []
    scaleparameter_times = []


    startdate    = pd.to_datetime(test_timeperiod[0])
    enddate      = pd.to_datetime(test_timeperiod[-1])
    startdate_dt = pd.to_datetime(startdate, format='%Y-%m-%d')
    enddate_dt   = pd.to_datetime(enddate,   format='%Y-%m-%d')
    starts_linspace_dt = pd.date_range(start=startdate_dt ,
                                         end=enddate_dt   ,
                                        freq=str(1)+"D")

    ### Make a list of the initial conditions for daily epoch start times that match
    ### the first instance of pce data for that day
    for iday, day in enumerate(starts_linspace_dt):

        epoch_start =  f"{day.strftime('%Y-%m-%d %H:%M:%S')}" #f"2018-11-{dayval:02d} 00:00:00"
        arc_startDT= pd.to_datetime(epoch_start,format='%Y-%m-%d %H:%M:%S')
        
        start_0ut = arc_startDT
        end___0ut = arc_startDT + pd.to_timedelta(24, 'h')
        epoch_endDT = end___0ut + pd.to_timedelta(3, 'h')
        

        epoch_startDTtest   = pd.to_datetime(epoch_start,format='%Y-%m-%d %H:%M:%S')-pd.to_timedelta(3, 'h')

        epoch_startDT = nearest( pd.to_datetime(xyzline['Date'].values), epoch_startDTtest)
        indx = np.where(pd.to_datetime(xyzline['Date'].values) == pd.to_datetime(epoch_startDT))[0][0]

        time_sep = np.abs((epoch_startDTtest - xyzline['Date'][indx]).total_seconds())/60
        
        print(' ')
        print(f" arc           {arc_startDT.strftime('%Y.%j')}")
        print(f" epoch_startDT {epoch_startDT}        {time_sep} mins from 21:00")
        print(f" start_0ut     {start_0ut} ")
        print(f" end___0ut     {end___0ut} ")
        print(f" epoch_endDT   {epoch_endDT} ")

        list_arc_startDT.append(arc_startDT.strftime('%Y.%j'))
        list_epoch_startDT.append(epoch_startDT)
        list_start_0ut.append(start_0ut)
        list_end___0ut.append(end___0ut)
        list_epoch_endDT.append(epoch_endDT)
    
    
        scalestart = str(pd.to_datetime(arc_startDT,format="%Y.%j"))
        scalestop  = str(pd.to_datetime(arc_startDT,format="%Y.%j") + pd.to_timedelta(24,'h'))
        scaletimes = pd.date_range(start = pd.to_datetime(scalestart),
                                   end   = pd.to_datetime(scalestop ),
                                   freq=str(scale_cadence)+"H")

        ###append the epoch end time to the end
        add_epochend = np.append(scaletimes.values.astype(np.int64) // 10 ** 9,
                      pd.to_datetime(pd.Series(epoch_endDT)).values.astype(np.int64) // 10 ** 9)
        scaleparameter_times.append([pd.to_datetime(datetime.strftime(datetime.fromtimestamp(ts), '%y%m%d%H%M%S.%f'),
            format ='%y%m%d%H%M%S.%f' )
                         for ts in add_epochend ])
        print(f" scaleparameter_times     {scaleparameter_times} ")

    return(list_arc_startDT,
           list_epoch_startDT,
           list_start_0ut,
           list_end___0ut,
           list_epoch_endDT,
           scaleparameter_times) 




#---------------------------------------------------------------------------------





Sat_List = ['spire083']  
# test_timeperiod = ["2018-11-02 00:00:00", "2018-11-29 00:00:00"]  # 4 days
test_timeperiod = ["2018-11-07 00:00:00", "2018-11-08 00:00:00"]  # 4 days


for isat,satval in enumerate(Sat_List):
    satnum = int(satval[5:])
    file_raw_ICs = f"/data/SatDragModelValidation/data/inputs/"\
              +f"sat_spire{satnum:03d}/g2b/Spire{satnum:03d}_RawEphem_20181101_20181130.txt"

    (arc_start, epoch_startDT, start_0ut,
    end___0ut, epoch_endDT, scaleparameter_times) = get_dates(test_timeperiod, file_raw_ICs)
    






 
 arc           2018.311
 epoch_startDT 2018-11-06 20:57:52        2.1333333333333333 mins from 21:00
 start_0ut     2018-11-07 00:00:00 
 end___0ut     2018-11-08 00:00:00 
 epoch_endDT   2018-11-08 03:00:00 
 scaleparameter_times     [[Timestamp('2018-11-07 00:00:00'), Timestamp('2018-11-07 03:00:00'), Timestamp('2018-11-07 06:00:00'), Timestamp('2018-11-07 09:00:00'), Timestamp('2018-11-07 12:00:00'), Timestamp('2018-11-07 15:00:00'), Timestamp('2018-11-07 18:00:00'), Timestamp('2018-11-07 21:00:00'), Timestamp('2018-11-08 00:00:00'), Timestamp('2018-11-08 03:00:00')]] 
 
 arc           2018.312
 epoch_startDT 2018-11-07 20:47:51        12.15 mins from 21:00
 start_0ut     2018-11-08 00:00:00 
 end___0ut     2018-11-09 00:00:00 
 epoch_endDT   2018-11-09 03:00:00 
 scaleparameter_times     [[Timestamp('2018-11-07 00:00:00'), Timestamp('2018-11-07 03:00:00'), Timestamp('2018-11-07 06:00:00'), Timestamp('2018-11-07 09:00:00'), Timestamp('2018-11-07 12:00:00'), Timestamp('2018-11-07 1

In [3]:
%load_ext autoreload
%autoreload 2
from gc import collect as gc_collect
import pickle 
import os
from datetime import datetime,timedelta


# g2b_path = "/data/SatDragModelValidation/data/inputs/sat_icesat2/g2b/"
g2b_path = "/data/SatDragModelValidation/data/inputs/sat_spire083/g2b/"
            
dir_save    =  '/data/SatDragModelValidation/data/outputs_clean/'\
             + 'spire083/O2R_comparisons/1_DRIAruns_3hr/'



obj = {}

for imonth,month in enumerate(month_list):
    if month=='oct':
        m_num = 10
    if month=='nov':
        m_num = 11
    if month=='dec':
        m_num = 12
    if month=='jan':
        m_num = 1
    if month=='feb':
        m_num = 2
    if month=='mar':
        m_num = 3
    if month=='apr':
        m_num = 4

#     file_raw_ICs = f"{g2b_path}ICESat2_RawEphem_2018_{m_num:02d}.txt"
#     file_g2b     = f"pce_icesat2_pso_2018_{m_num:02d}"
    file_raw_ICs = f"{g2b_path}Spire083_RawEphem_20181101_20181130.txt"
    file_g2b     = f"pce_spire083_leoOrb_20181101_20181130"

    
    
    for i,den in enumerate(run_list):
        settings_spire= {# Basic input settings
                     'satellite'      : {'input': 'spire083'},
                     'den_model'      : {'input': den},
                     'run_type'       : {'input': 'DataReduction_PCE'},
                     'run_specifier'  : {'input': '_'+month},
                     'cd_model'       : {'input': 'DRIA'},
                     'file_string'    : {'input': 'DRIAscaled_'},
                     'model_data_path' : {'input': run_dict[month+den]['model_path']},
                     'verbose' : {'input': True},
                     # Force Model settings
                      'cd_type'               : {'input':"Fixed_CD"},
                      'cd_value'              : {'input':1.0 },
                      'scaling_factor'        : {'input':True},
                      'hours_between_cd_adj'  : {'input':scaling_cadence},
                      #### Comment for unadjusted run:
                      'cd_adjustment_boolean' : {'input':True },
                    #### DRIA CD Model Parameters
                    'cd_model_params' : {'input':{ 
                            'MS'     : '26.980D0'   ,  #!  molar mass for each panel (g/mol)
                            'TW'     : '300.0D0'    ,  #!  temperature of panels  (K)
                            'ALPHA'  : '0.890D0'    ,  #!  accomodation coefficient, Alpha is b/w 0 and 1
                            'KL'     : '0.0D0'    ,    #!  langmuir parameter
                            'FRACOX' : '1.0D0'   ,     #!  fraction of surface covered by atomic oxygen
                       }},
                      #### ---------------------------------------
                     # Run
            
            
#             arc_startDT,
#             epoch_startDT,
# #             start_0ut,
# #             end___0ut,
#             epoch_endDT,
#             scaleparameter_times
                      'arc_type'       : {'input':'Nominal30hr_and_AB'},      
                      'step'           : {'input': 60.},
                      'orbfil_step'    : {'input': 120.},
                      'which_ICfile'   : {'input':file_raw_ICs},
                      'which_g2bfile'  : {'input':file_g2b},
                        #
                      'arc'            : {'input': arc_start},
                      'epoch_start'    : {'input': epoch_startDT},
                      'epoch_stop'     : {'input': epoch_endDT},  
#                 'scaleparameter_times' : {'input': scaleparameter_times},  
                'scaleparameter_times' : {'input': None},  
                       #                                
                      'global_options' : {'input':'pso_2018'},
                     # Request read on raw outputs
                      'request_data'   : {'input': ['Trajectory_orbfil', 
                                                   'Density', 
                                                   'Residuals_summary',
                                                   'DragFile',
                                                   'AdjustedParams'
                                                   ]},
                  #end dict
                  }

#         sat = Pygeodyn(settings_spire, use_file=False)
#         sat.run_arcs()
#         obj[month+den] =  sat.getData_BigData_lowmemory(orbit_propagation=False)
#         obj[month+den] = vars(obj[month+den])
#         gc_collect()

startdate = "2018-11-01"
enddate   = "2018-11-30"  #24

sat = Pygeodyn(settings_spire, use_file=False)
sat.initialize_timeperiod_stage1(startdate, enddate,
                             overwrite_exat=True, 
                             overwrite_ICtext=False)
sat = 0

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
spire083
check sat Spire
----------------------------------------------------------------------------
Initializing the time period from 2018-11-01 00:00:00 to 2018-12-01 00:00:00
     overwriting the epoch start and stop to match
----------------------------------------------------------------------------

self.prms['arc'] ['2018.305', '2018.306', '2018.307', '2018.308', '2018.309', '2018.310', '2018.311', '2018.312', '2018.313', '2018.314', '2018.315', '2018.316', '2018.317', '2018.318', '2018.319', '2018.320', '2018.321', '2018.322', '2018.323', '2018.324', '2018.325', '2018.326', '2018.327', '2018.328', '2018.329', '2018.330', '2018.331', '2018.332', '2018.333', '2018.334']
spire083
check sat Spire
Step 0: Make directory structure for satellite input data
Directory Exists:  /data/SatDragModelValidation/data/inputs/sat_spire083
Directory Exists:  /data/SatDragModelValidation/data/inputs/sat_spire0

KeyboardInterrupt: 

In [ ]:
arc_start

In [ ]:

    
#         pickleName = f'_{month}_DRIA_scale{scale_cadence}.pkl'

#         pickle_file = dir_save+den+pickleName
#         if not os.path.exists(pickle_file):
#             print('Must create pickle file...')
#             print('   ',  pickle_file)
#             print('   ', 'Reading Geodyn Data')

#             ### Load the data into an object
#             sat = Pygeodyn(settings_icesat2, use_file=False)
#             obj = sat.getData_BigData_lowmemory(orbit_propagation=False)
#             gc_collect()

#             #### Pickle the object to save it
#             print('   ', 'Saving pickle')
#             filehandler = open(pickle_file, 'wb') 
#             pickle.dump(vars(obj), filehandler)
#             filehandler.close()
#             obj = 0
#             print('   ', 'Saved pickle')

# obj = {}
# for i,model in enumerate(run_list):
#     for imonth,month in enumerate(month_list):
#         pickleName = f'_{month}_DRIA_scale{scale_cadence}.pkl'

#         ### Load the data if the pickles exist
#         print()
#         print()
#         gc_collect()

#         pickle_file = dir_save+model+pickleName

#         filehandler = open(pickle_file, 'rb') 
#         obj[month+model] = pickle.load(filehandler)
#         filehandler.close()
#         print('Loaded data from pickle... ',  model)
    
    
# ### Save space if doing density retrieval
# for model in run_dict.keys():
#     del obj[model]['OrbitResids']
#     del obj[model]['Trajectory_orbfil']
    
# gc_collect()